# Variational Auto-encoder

## Description of the algorithm

Let us consider a dataset $X = \{x^{(i)}\}_{i=1}^N$ consisting of N i.i.d. samples. We assume that the data are generated from parametric family of distributions $p_{\theta^*}(x)$ and we introduce the generative model $p_{\theta^*}(x, z) = p_{\theta^*}(x|z)p_{\theta^*}(z)$ where $z$ is an unobserved continuous random variable. The true parameters $\theta^*$ and the values of the latent variables $z^{(i)}$ are unknown to us. 

It is worth noting that we are interested in a general algorithm that works efficiently in the case of:
* intractability of the marginal likelihood $p_{\theta}(x) = \int p_{\theta}(x|z)p_{\theta}(z)dz$ and the true posterior density $p_{\theta}(z|x) = \dfrac{p_{\theta}(x|z)p_{\theta}(z)}{p_{\theta}(x)}$;
* scalability for a large dataset .

Our purpose is to solve the following three problems:
* efficient approximate ML or MAP estimation for the parameters $\theta$; 
* efficient approximate posterior inference of the latent variable $p_{\theta}(z|x)$; 
* efficient approximate marginal inference of the variable x. 

The algorithm which solves the above problems was proposed by D. Kingma and Prof. Dr. M. Welling in the paper [Auto-Encoding Variational Bayes](http://arxiv.org/abs/1312.6114). At first authors introduce a recognition model $q_{\varphi}(z|x)$: an approximation to the intractable true posterior $p_{\theta}(z|x)$. After Kingma et al. introduce a method for learning the recognition model parameters $\varphi$ jointly with the generative model parameters $\theta$. 

The key idea is to use the variational lower bound of the marginal likelihood 
$\ln p_{\theta}(x^{(1)}, \dots, x^{(N)})= \sum_{i=1}^N\ln p_{\theta}(x^{(i)})$:

$$
\ln p_{\theta}(x^{(i)}) = D_{KL}(q_{\varphi}(z|x^{(i)})||p_{\theta}(z|x^{(i)})) + L(\theta, \varphi; x^{(i)}) \quad \Rightarrow 
$$
$$
\Rightarrow \quad \ln p_{\theta}(x^{(i)}) \geqslant L(\theta, \varphi; x^{(i)}) = \mathbf{E}_{q_{\varphi}(z|x)}\left(-\ln q_{\varphi}(z|x) + \ln p_{\theta}(x, z)\right) = -D_{KL}(q_{\varphi}(z|x^{(i)})||p_{\theta}(z)) + \mathbf{E}_{q_{\varphi}(z|x^{(i)})}(\ln p_{\theta}(x^{(i)}|z))
$$


Our aim is to maximize the lower bound $L(\theta, \varphi; x^{(i)})$ w.r.t. both the variational parameters $\varphi$ and the generative parameters $\theta$. However, we have some difficulties with the gradient of the lower bound w.r.t. $\varphi$. The usual Monte Carlo gradient estimator is:

$$
\nabla_{\varphi}\mathbf{E}_{q_{\varphi}(z)}\left[f(z, \varphi)\right] = \mathbf{E}_{q_{\varphi}(z)}
\left[\nabla_{\varphi}f(z, \varphi)\right] + \mathbf{E}_{q_{\varphi}(z)}\left[f(z, \varphi)\nabla_{\varphi}\ln q_{\varphi}(z)\right] \approx
\dfrac{1}{L}\sum_{l=1}^L(\nabla_{\varphi}f(z_l, \varphi) + f(z_l, \varphi)\nabla_{\varphi}\ln q_{\varphi}(z_l))\quad
\text{ where } \quad z_l \sim q_{\varphi}(z)
$$

Unfortunately, the term $f(z_l, \varphi)\nabla_{\varphi}\ln q_{\varphi}(z_l)$ in our gradient estimator exhibits very high variance and is impractical for our purposes. But in case of the continuous latent variable $z$ with certain mild conditions for a chosen approximate posterior $q_{\theta}(z|x)$ we can utilize the reparameterization trick. The idea is simple. If it is possible to express the variable $z$ as a deterministic variable $z = g_{\varphi}(\varepsilon, x)$ where $\varepsilon$ is a random variable with independent marginal $p(\varepsilon)$ and $g_{\varphi}(.)$ is some vector-valued function parameterized by $\varphi$, then the following is true: 

$$
\int q_{\varphi}(z|x)f(z, \varphi)dz = \int p(\varepsilon)f(z, \varphi) d\varepsilon = \int p(\varepsilon)f(g_{\varphi}(\varepsilon, x), \varphi)d\varepsilon
$$

So we obtain more robust Monte Carlo gradient estimator:

$$
\nabla_{\varphi}\mathbf{E}_{q_{\varphi}(z|x)}\left[f(z, \varphi)\right] = \nabla_{\varphi}\mathbf{E}_{p(\varepsilon)}\left[f(g_{\varphi}(\varepsilon, x), \varphi)\right] = \mathbf{E}_{p(\varepsilon)}\left[\nabla_{\varphi}f(g_{\varphi}(\varepsilon, x), \varphi)\right] \approx
\dfrac{1}{L}\sum_{l=1}^L\nabla_{\varphi}f(g_{\varphi}(\varepsilon_l, x), \varphi)\quad
\text{ where } \quad \varepsilon_l \sim p(\varepsilon)
$$

We apply this technique to the variational lower bound, yielding the generic Stochastic Gradient Variational Bayes (SGVB) estimator $\hat{L}(\theta, \varphi; x) \approx L(\theta, \varphi; x)$:

$$
\hat{L}(\theta, \varphi; x) = \dfrac{1}{L}\sum_{i=1}^L\ln p_{\theta}(x, z_l) - \ln q_{\varphi}(z_l|x) \quad
\text{ where } \quad z_l = g_{\varphi}(\varepsilon_l, x) \quad \text{ and } \quad \varepsilon_l \sim p(\varepsilon). 
$$

In [1]:
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(0)
tf.set_random_seed(0)

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def xavier_init(fan_in, fan_out, constant=1): 
    low = -constant*np.sqrt(6.0/(fan_in + fan_out)) 
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), 
                             minval=low, maxval=high, 
                             dtype=tf.float32)

In [18]:
class VariationalAutoencoder(object):
    def __init__(self, n_input, n_z, network_architecture, 
                 learning_rate=0.001, decoder_distribution='gaussian'):
        self.n_input = n_input
        self.n_z = n_z
        self.network_architecture = network_architecture
        self.learning_rate = learning_rate
        self.decoder_distribution = decoder_distribution
        
        self.x = tf.placeholder(tf.float32, [None, n_input])
        
        self._create_network()
        
        self._create_loss_optimizer()
        
        init = tf.initialize_all_variables()
        self.sess = tf.Session()
        self.sess.run(init)
        
    def _create_network(self):
        self.weights = self._initialize_weights(**self.network_architecture)
        
        encoder_layer1 = tf.nn.softplus(tf.add(tf.matmul(self.x, self.weights['encoder']['h1']),
                                               self.weights['encoder']['b1']))
        encoder_layer2 = tf.nn.softplus(tf.add(tf.matmul(encoder_layer1, self.weights['encoder']['h2']), 
                                               self.weights['encoder']['b2']))
        self.z_mean = tf.add(tf.matmul(encoder_layer2, self.weights['encoder']['out_mean']), 
                             self.weights['encoder']['out_mean_b'])
        self.z_log_sigma_sq = tf.add(tf.matmul(encoder_layer2, self.weights['encoder']['out_log_sigma_sq']), 
                                     self.weights['encoder']['out_log_sigma_sq_b'])
        epsilon = tf.random_normal((tf.shape(self.x)[0], self.n_z), 0, 1, dtype=tf.float32)
        self.z = tf.add(self.z_mean, tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq)), epsilon))
        
        decoder_layer1 = tf.nn.softplus(tf.add(tf.matmul(self.z, self.weights['decoder']['h1']),
                                               self.weights['decoder']['b1']))
        decoder_layer2 = tf.nn.softplus(tf.add(tf.matmul(decoder_layer1, self.weights['decoder']['h2']), 
                                               self.weights['decoder']['b2']))
        self.x_reconstruction = tf.sigmoid(tf.add(tf.matmul(decoder_layer2, self.weights['decoder']['out_mean']),
                                                  self.weights['decoder']['out_mean_b']))
        
    def _initialize_weights(self, n_hidden_encoder_1, n_hidden_encoder_2, 
                           n_hidden_decoder_1, n_hidden_decoder_2):
        weights = dict()
        weights['encoder'] = {
            'h1': tf.Variable(xavier_init(self.n_input, n_hidden_encoder_1)), 
            'h2': tf.Variable(xavier_init(n_hidden_encoder_1, n_hidden_encoder_2)), 
            'out_mean': tf.Variable(xavier_init(n_hidden_encoder_2, self.n_z)),
            'out_log_sigma_sq': tf.Variable(xavier_init(n_hidden_encoder_2, self.n_z)),
            
            'b1': tf.Variable(tf.zeros([n_hidden_encoder_1], dtype=tf.float32)), 
            'b2': tf.Variable(tf.zeros([n_hidden_encoder_2], dtype=tf.float32)), 
            'out_mean_b': tf.Variable(tf.zeros([self.n_z], dtype=tf.float32)),
            'out_log_sigma_sq_b': tf.Variable(tf.zeros([self.n_z], dtype=tf.float32))
        }
        weights['decoder'] = {
            'h1': tf.Variable(xavier_init(self.n_z, n_hidden_decoder_1)), 
            'h2': tf.Variable(xavier_init(n_hidden_decoder_1, n_hidden_decoder_2)), 
            'out_mean': tf.Variable(xavier_init(n_hidden_encoder_2, self.n_input)),
            
            'b1': tf.Variable(tf.zeros([n_hidden_decoder_1], dtype=tf.float32)), 
            'b2': tf.Variable(tf.zeros([n_hidden_decoder_2], dtype=tf.float32)), 
            'out_mean_b': tf.Variable(tf.zeros([self.n_input], dtype=tf.float32))
        }
        return weights
    
    def _create_loss_optimizer(self):
        if self.decoder_distribution == 'gaussian':
            self.decoder_cost = 0.5 * tf.reduce_sum(tf.square(tf.sub(self.x_reconstruction, self.x)), 1)
        elif self.decoder_distribution == 'bernoulli':
            self.decoder_cost = -tf.reduce_sum(self.x * tf.log(1e-10 + self.x_reconstruction)
                                               + (1-self.x) * tf.log(1e-10 + 1 - self.x_reconstruction), 1)
        else:
            raise ValueError('Unsupported decoder distribution!')
        self.encoder_cost = -0.5 * tf.reduce_sum(1 + self.z_log_sigma_sq 
                                            - tf.square(self.z_mean) 
                                            - tf.exp(self.z_log_sigma_sq), 1)
        self.cost = tf.reduce_mean(self.decoder_cost + self.encoder_cost)
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
        
    def partial_fit(self, X):
        opt, cost = self.sess.run((self.optimizer, self.cost), 
                                  feed_dict={self.x: X})
        return cost
    
    def transform(self, X):
        return self.sess.run(self.z_mean, feed_dict={self.x: X})
    
    def generate(self, z=None):
        if z is None:
            z = np.random.normal(size=self.n_z)
        return self.sess.run(self.x_reconstruction, feed_dict={self.z: z})
    
    def reconstruct(self, X):
        return self.sess.run(self.x_reconstruction, feed_dict={self.x: X})
    
    def loss(self, X):
        return self.sess.run(self.cost, feed_dict={self.x: X})
    
    def decoder_loss(self, X):
        return self.sess.run(self.decoder_cost, feed_dict={self.x: X})
    
    def encoder_loss(self, X):
        return self.sess.run(self.encoder_cost, feed_dict={self.x: X})

In [5]:
n_samples = mnist.train.num_examples
n_input = 784
n_z = 20
batch_size = 100
learning_rate = 0.001

network_architecture = {
    'n_hidden_encoder_1': 500,
    'n_hidden_encoder_2': 500,
    'n_hidden_decoder_1': 500,
    'n_hidden_decoder_2': 500
}

In [21]:
def train(data, n_samples, n_input, n_z, batch_size, 
          network_architecture, learning_rate, decoder_distribution,
          training_epochs=10, display_step=5):
    vae = VariationalAutoencoder(n_input, n_z, network_architecture, 
                                 learning_rate, decoder_distribution)
    for epoch in xrange(training_epochs):
        avg_cost = 0.
        total_batch = int(n_samples / batch_size)
        for i in xrange(total_batch):
            batch_xs, _ = data.train.next_batch(batch_size)
            vae.partial_fit(batch_xs)
            cost = vae.loss(batch_xs)
            avg_cost += cost / n_samples * batch_size
        
        if epoch % display_step == 0:
            print('Epoch: {:04d}, cost = {:.9f}, test cost = {:.9f}' \
                  .format(epoch+1, avg_cost, vae.loss(data.test.images)))
    return vae

### Training in Gaussian case

In [8]:
vae = train(mnist, n_samples, n_input, n_z, 
            batch_size, network_architecture, 
            learning_rate, 'gaussian', training_epochs=75)

Epoch: 0001, cost = 26.830678402, test cost = 26.169908524
Epoch: 0006, cost = 21.699161960, test cost = 21.548978806
Epoch: 0011, cost = 20.347310236, test cost = 20.237585068
Epoch: 0016, cost = 19.873470220, test cost = 19.883232117
Epoch: 0021, cost = 19.648401035, test cost = 19.668369293
Epoch: 0026, cost = 19.455578367, test cost = 19.484958649
Epoch: 0031, cost = 19.262910617, test cost = 19.314926147
Epoch: 0036, cost = 19.053380540, test cost = 19.132003784
Epoch: 0041, cost = 18.906792235, test cost = 19.012399673
Epoch: 0046, cost = 18.818410835, test cost = 18.907796860
Epoch: 0051, cost = 18.739035298, test cost = 18.829015732
Epoch: 0056, cost = 18.667027668, test cost = 18.784650803
Epoch: 0061, cost = 18.586155396, test cost = 18.728923798
Epoch: 0066, cost = 18.539185541, test cost = 18.650930405
Epoch: 0071, cost = 18.467723257, test cost = 18.659910202


### Training in Bernoulli case

In [20]:
vae = train(mnist, n_samples, n_input, n_z, 
            batch_size, network_architecture, 
            learning_rate, 'bernoulli', training_epochs=75)

Epoch: 0001, cost = 174.010542797, test cost = 139.077346802
Epoch: 0006, cost = 108.510390348, test cost = 107.655998230
Epoch: 0011, cost = 103.943840166, test cost = 104.144309998
Epoch: 0016, cost = 101.791891369, test cost = 102.287849426
Epoch: 0021, cost = 100.329291770, test cost = 101.443397522
Epoch: 0026, cost = 99.015549094, test cost = 100.691474915
Epoch: 0031, cost = 98.122421639, test cost = 99.742271423
Epoch: 0036, cost = 97.364934665, test cost = 99.391883850
Epoch: 0041, cost = 96.791749337, test cost = 98.885337830
Epoch: 0046, cost = 96.290269567, test cost = 98.671485901
Epoch: 0051, cost = 95.868905986, test cost = 98.590072632
Epoch: 0056, cost = 95.555079054, test cost = 98.299934387
Epoch: 0061, cost = 95.175326968, test cost = 97.915428162
Epoch: 0066, cost = 94.922360243, test cost = 97.979713440
Epoch: 0071, cost = 94.625248774, test cost = 97.754158020
